In [62]:
# coding:u8
import pandas as pd
import numpy as np
import copy,gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score,classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
import seaborn as sns



In [63]:
data = pd.read_pickle('features/1_stat_feature.pkl')

In [64]:
data = data.fillna(-999)

In [65]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,rank_num_attrs,rank_num_abstract_words,rank_num_alias,rank_label_mean,rank_label_count,rank_m_label_mean,rank_m_label_count,rank_num_candidates,rank_mention_equal_subject,rank_shuminghao
0,1,311223,南京南站,1,17,447,3,0,2,True,...,1.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,1.5,1.5,1.5
1,1,130287,南京南站,0,9,338,2,0,2,True,...,2.0,2.0,2.0,-999.0,-999.0,-999.0,-999.0,1.5,1.5,1.5
2,1,213561,高铁,0,1,0,2,1,2,False,...,2.0,2.0,1.0,2.0,1.5,2.0,1.5,1.5,2.0,1.5
3,1,341096,高铁,1,10,239,1,1,2,True,...,1.0,1.0,2.0,1.0,1.5,1.0,1.5,1.5,1.0,1.5
4,1,311223,南京南站,1,17,447,3,2,2,True,...,1.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,1.5,1.5,1.5


In [68]:

predictor = ['num_attrs',
       'num_abstract_words', 'num_alias', 'num_candidates',
       'mention_equal_subject', 'shuminghao', 'len_mention', 'mention_start',
       'academicdiscipline', 'animal', 'astronomicalobject', 'athlete',
       'awardeventseries', 'brand', 'building', 'collegeoruniversity',
       'communicationmedium', 'country', 'creativework', 'culturalheritage',
       'currency', 'curriculum', 'educationmajor', 'entertainmentperson',
       'event', 'familyname', 'fictionalhuman', 'fictionalthing', 'food',
       'formula', 'game', 'historicalperiod', 'historicalperson', 'human',
       'internationalorganization', 'language', 'material', 'medicalcondition',
       'medicaldepartmenttype', 'movie', 'nation', 'organism', 'organization',
       'person', 'place', 'plant', 'product', 'scientificorganization',
       'symbol', 'theorem', 'thing', 'tool', 'tvplay', 'tvshow', 'vocabulary',
       'zodiacsign', 'label_mean', 'label_count', 'm_label_mean',
       'm_label_count', 'rank_num_attrs', 'rank_num_abstract_words',
       'rank_num_alias', 'rank_label_mean', 'rank_label_count',
       'rank_m_label_mean', 'rank_m_label_count', 'rank_num_candidates',
       'rank_mention_equal_subject', 'rank_shuminghao']

In [69]:
exclude = []
# exclude = []
temp = []
for x in predictor:
    if x not in exclude:temp.append(x)
predictor2 = temp

In [70]:
param = {
    #'num_leaves':100,
    'objective':'binary',
    #'metric':'acc',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    'colsample_bytree':0.8,
    'n_estimators':20000,
    'min_child_weight':1,
    'subsample_freq':2,
    'num_leaves':128,
    'n_jobs':-1,
}


In [71]:
kfold = KFold(n_splits=9,shuffle=False,random_state=2019)
for train_index,test_index in kfold.split(data):
    train_data,val_data = data.loc[train_index,:],data.loc[test_index,:]
    train = lgb.Dataset(train_data[predictor2],label=train_data['label'])
    validation_data = lgb.Dataset(val_data[predictor2],label=val_data['label'])
    
    val_data.reset_index(inplace=True,drop=True)
    val_data.reset_index(inplace=True)
    index_df = val_data.groupby('m_id')['index'].apply(list)
    
    label_package = val_data.groupby('m_id',as_index=False)['label'].agg({'label_id_max':'idxmax'})['label_id_max']
    
    def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        hit = 0
        for index,pack in enumerate(index_df):
            score = []
            for i in pack:
                score.append(y_hat[i])
            if pack[np.argmax(score)]==label_package[index]:
                hit+=1
        best_score = hit/len(index_df)
        return 'acc', best_score, True
    bst = lgb.train(param, train, valid_sets=[validation_data],early_stopping_rounds=500,
                    verbose_eval=20,feval=lgb_f1_score)
#     break
# 0.8672
# 0.872
# [171]	valid_0's binary_logloss: 0.0988336	valid_0's acc: 0.873219
# valid_0's binary_logloss: 0.0964686	valid_0's acc: 0.87713
# [224]	valid_0's binary_logloss: 0.096617	valid_0's acc: 0.877515
# [308]	valid_0's binary_logloss: 0.09149	valid_0's acc: 0.890507 entity common
# drop ctr 0.877
# [2094]	valid_0's binary_logloss: 0.0791409	valid_0's acc: 0.896452
# [1674]	valid_0's binary_logloss: 0.0784442	valid_0's acc: 0.896347
#０．９１５

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 500 rounds.
[20]	valid_0's binary_logloss: 0.157503	valid_0's acc: 0.888948
[40]	valid_0's binary_logloss: 0.105108	valid_0's acc: 0.889652
[60]	valid_0's binary_logloss: 0.0883995	valid_0's acc: 0.890919
[80]	valid_0's binary_logloss: 0.082556	valid_0's acc: 0.891904
[100]	valid_0's binary_logloss: 0.0803302	valid_0's acc: 0.892784
[120]	valid_0's binary_logloss: 0.0793649	valid_0's acc: 0.893383
[140]	valid_0's binary_logloss: 0.0789201	valid_0's acc: 0.893629
[160]	valid_0's binary_logloss: 0.0786494	valid_0's acc: 0.893594
[180]	valid_0's binary_logloss: 0.0784948	valid_0's acc: 0.893875
[200]	valid_0's binary_logloss: 0.0783741	valid_0's acc: 0.893981
[220]	valid_0's binary_logloss: 0.0783051	valid_0's acc: 0.894333
[240]	valid_0's binary_logloss: 0.0781914	valid_0's acc: 0.894157
[260]	valid_0's binary_logloss: 0.0781102	valid_0's acc: 0.894263
[280]	valid_0's binary_logloss: 0.0780461	valid_0's acc: 0.894439
[300]	valid_0's bina

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 500 rounds.
[20]	valid_0's binary_logloss: 0.156436	valid_0's acc: 0.889167
[40]	valid_0's binary_logloss: 0.103792	valid_0's acc: 0.889909
[60]	valid_0's binary_logloss: 0.0870845	valid_0's acc: 0.890509
[80]	valid_0's binary_logloss: 0.0811983	valid_0's acc: 0.891427
[100]	valid_0's binary_logloss: 0.0789214	valid_0's acc: 0.892169
[120]	valid_0's binary_logloss: 0.077979	valid_0's acc: 0.892698
[140]	valid_0's binary_logloss: 0.0774282	valid_0's acc: 0.892875
[160]	valid_0's binary_logloss: 0.0771908	valid_0's acc: 0.892804
[180]	valid_0's binary_logloss: 0.0770143	valid_0's acc: 0.893193
[200]	valid_0's binary_logloss: 0.0768949	valid_0's acc: 0.893475
[220]	valid_0's binary_logloss: 0.0767608	valid_0's acc: 0.893616
[240]	valid_0's binary_logloss: 0.0766801	valid_0's acc: 0.89344
[260]	valid_0's binary_logloss: 0.0765943	valid_0's acc: 0.893722
[280]	valid_0's binary_logloss: 0.0765333	valid_0's acc: 0.893404
[300]	valid_0's binar

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 500 rounds.
[20]	valid_0's binary_logloss: 0.156043	valid_0's acc: 0.88832
[40]	valid_0's binary_logloss: 0.103511	valid_0's acc: 0.889494
[60]	valid_0's binary_logloss: 0.0866554	valid_0's acc: 0.890739
[80]	valid_0's binary_logloss: 0.0806412	valid_0's acc: 0.891415
[100]	valid_0's binary_logloss: 0.0783448	valid_0's acc: 0.891735
[120]	valid_0's binary_logloss: 0.0773166	valid_0's acc: 0.892304
[140]	valid_0's binary_logloss: 0.0767622	valid_0's acc: 0.893087
[160]	valid_0's binary_logloss: 0.0764993	valid_0's acc: 0.893443
[180]	valid_0's binary_logloss: 0.0763411	valid_0's acc: 0.89355
[200]	valid_0's binary_logloss: 0.0762353	valid_0's acc: 0.893728
[220]	valid_0's binary_logloss: 0.0761553	valid_0's acc: 0.893692
[240]	valid_0's binary_logloss: 0.0760743	valid_0's acc: 0.893941
[260]	valid_0's binary_logloss: 0.0759655	valid_0's acc: 0.893977
[280]	valid_0's binary_logloss: 0.0758895	valid_0's acc: 0.894012
[300]	valid_0's binar

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 500 rounds.
[20]	valid_0's binary_logloss: 0.159493	valid_0's acc: 0.88686
[40]	valid_0's binary_logloss: 0.1067	valid_0's acc: 0.8881
[60]	valid_0's binary_logloss: 0.0899887	valid_0's acc: 0.889168
[80]	valid_0's binary_logloss: 0.0840793	valid_0's acc: 0.889754
[100]	valid_0's binary_logloss: 0.0818646	valid_0's acc: 0.890546
[120]	valid_0's binary_logloss: 0.0809254	valid_0's acc: 0.890788
[140]	valid_0's binary_logloss: 0.0804955	valid_0's acc: 0.891373
[160]	valid_0's binary_logloss: 0.0802338	valid_0's acc: 0.891442
[180]	valid_0's binary_logloss: 0.0800739	valid_0's acc: 0.891511
[200]	valid_0's binary_logloss: 0.079954	valid_0's acc: 0.89158
[220]	valid_0's binary_logloss: 0.0798416	valid_0's acc: 0.891821
[240]	valid_0's binary_logloss: 0.0797644	valid_0's acc: 0.892131
[260]	valid_0's binary_logloss: 0.0796862	valid_0's acc: 0.8922
[280]	valid_0's binary_logloss: 0.0796308	valid_0's acc: 0.892235
[300]	valid_0's binary_loglo

KeyboardInterrupt: 

In [ ]:
############################## for submit  #####################################

train = lgb.Dataset(data[predictor2],label=data['label'])
param = {
    #'num_leaves':100,
    'objective':'binary',
    #'metric':'acc',
    'metric_freq':20,
    'learning_rate': 0.05,
    'num_threads':8,
    #'min_sum_hessian_in_leaf':10,
    'boosting_type':'gbdt',
    'subsample':0.9,
    'colsample_bytree':0.8,
    'n_estimators':1000,
    'min_child_weight':1,
    'subsample_freq':2,
    'num_leaves':128,
    'n_jobs':-1,
}

bst = lgb.train(param, train)
from sklearn.externals import joblib
joblib.dump(bst, 'model_link/lgb.pkl')

# save model
# load model
#gbm_pickle = joblib.load('lgb.pkl')
# break

In [ ]:
# val_data

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:
fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = predictor2
fold_importance_df["importance"] = bst.feature_importance()
cols = (fold_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')